In [1]:
import petl as etl
import psycopg2 as pg
import pandas as pd
import sys
from sqlalchemy import *

In [2]:
conn = pg.connect(host="localhost", database="qtsurvey_db_FMSP", user="postgres", password="postgres")
# Open a cursor to perform database operations
cur = conn.cursor()

In [3]:
ue = 622 # Pharmacie clinique

In [4]:
# Execute a query
#driving_distance module
#note the lack of trailing semi-colon in the query string, as per the Postgres documentation
query = """ SELECT id_resultat_evaluation, fiche_evaluation, question, public.question.valeur_question ,reponse FROM public.resultatevaluation
        INNER JOIN public.question ON public.question.id_question = public.resultatevaluation.question 
        where  fiche_evaluation IN (SELECT id_fiche_evaluation  FROM public.ficheevaluation where ue= 622 order by 1) order by fiche_evaluation, question Asc """

In [5]:
outputquery = "COPY ({0}) TO STDOUT WITH CSV HEADER".format(query)
filename = 'ue-'+ str( ue ) +'_2021-2022.csv'
filename2 = 'ue-'+ str( ue ) +'_2021-2022output.csv'

In [6]:
with open(filename, 'w') as f:
    cur.copy_expert(outputquery, f)
conn.close()

In [7]:
# Reading the csv file and storing it
# in a variable
df = pd.read_csv(filename);
df.tail()

,id_resultat_evaluation,fiche_evaluation,question,valeur_question,reponse
125,16234,5857,37,Possibilité d'échange avec l'enseignant en deh...,3
126,16235,5857,38,Conformité des évaluations aux modalités annon...,4
127,16236,5857,39,Niveau de couverture du contenu du cours,2
128,16237,5857,40,Disponibilité du corrigé des contrôles continus,4
129,16238,5857,41,Appréciation générale à la fin des cours,5


In [8]:
crosstb = df.pivot_table(index='fiche_evaluation', columns='valeur_question', values='reponse', aggfunc='max')
crosstb.tail()

valeur_question,Appréciation générale à la fin des cours,Conformité des évaluations aux modalités annoncées,Création d'une atmosphère favorable,Disponibilité du corrigé des contrôles continus,Disponibilité du plan du cours,Indication d'une bibliographie sélective,Niveau de couverture du contenu du cours,Possibilité d'échange avec l'enseignant en dehors des cours,Présentation claire et précise des objectifs du cours,Présentation des modalités d'évaluation,Respect des horaires et ponctualité,Respect du planning et des rythmes des cours,Utilisation des outils pédagogiques
fiche_evaluation,,,,,,,,,,,,,
5758,5,2,3,7,7,7,2,6,7,7,2,3,5
5769,4,5,2,3,6,4,4,4,5,6,3,3,3
5802,3,5,6,4,6,6,6,5,6,6,3,4,5
5824,3,5,2,4,5,5,7,5,5,5,3,4,5
5857,5,4,3,4,6,3,2,3,4,4,3,4,2


In [9]:
crosstb.to_csv(filename2, encoding='utf-8')